<a href="https://colab.research.google.com/github/Chandana-0812/DS_Feb12/blob/main/ANN_Basic_C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [105]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import sklearn.preprocessing as preprocessing
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score

# Step 1: Read Excel file
df = pd.read_excel('Final_Data_Absorption_21.xlsx')


# Select the first 37 columns
A = df.iloc[:, 0:16]

# Get the dimensions of A
na1, na2 = A.shape

# Descriptor Matrix for catalysts D1(1,5): ATOMIC NUMBERS
D1 = np.array([26, 40, 25, 28, 58, 39, 27, 22])

# Assigning of values from A to A1 matrix
A1 = np.zeros((na1, 13))  # total inputs
A1[:,7:13] = A.iloc[:,9:15]  # Operating param
target1 = A.iloc[:, 15]  # H2
# target1=A(:,31);  %CO
# target1=A(:,32); %CO2
# target1=A(:,33); %CH4

# METAL1
for i in range(na1):
    count = 0
    j1 = []
    for j in range(8):
        if A.iloc[i, j] != 0:
            count += 1
            j1.append(j)
    for j2 in range(count):
        A1[i, (2 * j2)] = D1[j1[j2]]  # Active metal
        A1[i, (2 * j2 + 1)] = A.iloc[i, j1[j2]]  # Active metal comp
        for j2 in range(3):
            if A1[i, (2 * j2)] == 0:  # Catalyst Composition if zero
                A1[i, (2 * j2)] = np.min(D1)
                A1[i, (2 * j2 + 1)] = 0

for i in range(na1):
    for j in range(9):
        if A.iloc[i, j] == 100:
            for j2 in range(3):
                A1[i, (2 * j2)] = np.min(D1)  # No Catalyst
                A1[i, (2 * j2 + 1)] = 0  # Its Composition

na1, na3 = A1.shape
print(f'Total number of observations: {na1}')
print(f'Total number of inputs/features: {na3}')
null_indices = np.isnan(A1)


Total number of observations: 791
Total number of inputs/features: 13


In [110]:
import numpy as np
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, r2_score
A1 = pd.DataFrame(A1)
# Load your data into InputData and TargetData
InputData = A1.iloc[:, 0:13]
TargetData = A.iloc[:, 15]

# Transpose the input and target data
x = InputData
t = TargetData

# Choose a Training Function
trainFcn = 'adam'  # Adam
np.random.seed(42)
# Create a Fitting Network
hiddenLayerSize = (5,)  # 4 nodes in one hidden layer
net = MLPRegressor(hidden_layer_sizes=hiddenLayerSize, activation='tanh', solver=trainFcn,
                   max_iter=5000, learning_rate_init=0.005, momentum=0.1, alpha=1e-10)

# Split data into training and testing sets
x_train, x_test, t_train, t_test = train_test_split(x, t, test_size=0.30, random_state=1)
# Min & Max values for inputs and outputs
mnx_train= np.min(x_train, axis=0)
mxx_train= np.max(x_train, axis=0)

print('mnx_train',mnx_train)
print('mxx_train',mxx_train)

mnx_test=np.min(x_test, axis=0)
mxx_test=np.max(x_test, axis=0)

mnt_train=min(t_train)
mxt_train=max(t_train)

mnt_test=min(t_test)
mxt_test=max(t_test)

na1_tr = A1.shape[0]  # Assuming na1_tr is the number of rows in B1
na3_tr = A1.shape[1]  # Assuming na3_tr is the number of columns in B1

# Input scaling
X_train = ((x_train - mnx_train) / (mxx_train - mnx_train)) * 10.0 - 5.0
X_test= ((x_test - mnx_test) / (mxx_test - mnx_test)) * 10.0 - 5.0

# Normalize Output Values
t_train=((t_train - mnt_train)/(mxt_train - mnt_train))*2-1
t_test=((t_test - mnt_test)/(mxt_test - mnt_test))*2-1

# Normalize data
#sc = MinMaxScaler()
#X_train = sc.fit_transform(x_train)
#X_test = sc.transform(x_test)

# Train the network
net.fit(X_train, t_train)

# Make predictions on training and testing data
y_train = net.predict(X_train)
y_test = net.predict(X_test)

# Denormalize output data
y_train = ((mnt_train + mxt_train) + y_train*(mxt_train - mnt_train))/2
y_test = ((mnt_test + mxt_test) + y_test*(mxt_test - mnt_test))/2

t_train=((mnt_train + mxt_train) + t_train*(mxt_train - mnt_train))/2
t_test=((mnt_test + mxt_test) + t_test*(mxt_test - mnt_test))/2

# Calculate performance measures for training data
RMSE_train = np.sqrt(mean_squared_error(t_train, y_train))
Rsq_train = r2_score(t_train, y_train)
AAD_train = np.mean(np.abs(y_train - t_train))

# Calculate performance measures for testing data
RMSE_test = np.sqrt(mean_squared_error(t_test, y_test))
Rsq_test = r2_score(t_test, y_test)
AAD_test = np.mean(np.abs(y_test - t_test))

# Print performance measures
print("Performance measures for training data:")
print("RMSE:", RMSE_train)
print("R-squared:", Rsq_train)
print("AAD:", AAD_train)
print("\nPerformance measures for testing data:")
print("RMSE:", RMSE_test)
print("R-squared:", Rsq_test)
print("AAD:", AAD_test)

mnx_train 0      26.0
1       0.6
2      22.0
3       0.0
4      22.0
5       0.0
6       0.0
7       9.0
8       5.0
9       2.0
10      0.0
11    100.0
12     12.0
dtype: float64
mxx_train 0        58.000000
1        10.000000
2        58.000000
3        11.400000
4        39.000000
5         4.600000
6         0.000000
7        50.000000
8     50000.000000
9        60.000000
10      144.189526
11      350.000000
12       40.000000
dtype: float64


ValueError: Input X contains NaN.
MLPRegressor does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
#Predicting result for Single Observation
xnew=[26, 2.5,	28,	5,	22,	2.5,	90,	45,	37,	8, 5,	250,	12]
print(net.predict(sc.transform([xnew])))

In [ ]:
# Load pre-trained weights
#weights = net.coefs_
# Extract weights
input_hidden_weights = net.coefs_[0]  # Weights between input and hidden layer
hidden_output_weights = net.coefs_[1]  # Weights between hidden and output layer

# Print weights
print("Input to Hidden Weights:")
print(input_hidden_weights)
print("\nHidden to Output Weights:")
print(hidden_output_weights)

# Print Bias terms
# Extract biases
hidden_biases = net.intercepts_[0]  # Biases for the hidden layer
output_bias = net.intercepts_[1]    # Bias for the output layer

print("Input to Hidden Bias:")
print(hidden_biases)
print("\nHidden to Output Bias:")
print(output_bias)

# Print or further process the weights
#print(weights)

In [ ]:
%whos
